# Named Entity Recognition using LSTM

### 1. Preparing vocabulary for Word Embeddings

In [40]:
import pandas as pd
import numpy as np
from textblob import TextBlob

In [2]:
df = pd.read_csv('./data/ner_dataset.csv')

In [16]:
words = df['Word'].str.lower()
words = words.drop_duplicates().values
print(words)
print(words.shape)

['thousands' 'of' 'demonstrators' ..., 'indicative' '3700' 'bermel']
(31804,)


In [17]:
word_indices = {}

In [21]:
def map_word_indices(words):
    for i in range(len(words)):
        word_indices[words[i]] = i

In [22]:
map_word_indices(words)

In [45]:
def get_index_matrix(s):
    id_matrix = []
    tb = TextBlob(s)
    w = tb.words
    w = [i.lower() for i in w]
    for i in w:
        if i in words:
            id_matrix.append(word_indices[i])
        else :
            id_matrix.append(31805) #Unknown token
    return id_matrix

In [54]:
sentence = "My name is Dumbledore"
print(get_index_matrix(sentence))

[1396, 2960, 75, 31805]


Generating this ID matrix will be useful for training the embedding layer.